# 🚀 Forex Trading System - PPO Training

**Goal**: Train AI to make $100 profit in 4 candles (1 hour)

**Algorithm**: PPO with Gemma 2B model  
**Duration**: 12-24 hours on GPU  
**Target**: 500k timesteps

---

## 🔧 1. Environment Setup

In [ ]:
# Check GPU availability
import torch
import os

if torch.cuda.is_available():
    gpu_name = torch.cuda.get_device_name(0)
    gpu_memory = torch.cuda.get_device_properties(0).total_memory / 1e9
    print(f"🎮 GPU Available: {gpu_name}")
    print(f"   Memory: {gpu_memory:.1f} GB")
else:
    print("⚠️ No GPU detected - training will be very slow!")
    print("Go to Runtime → Change Runtime Type → Hardware Accelerator → GPU")

print(f"\n📁 Current directory: {os.getcwd()}")
print(f"📂 Files available: {os.listdir()}")

In [ ]:
# Create necessary directories
!mkdir -p src config training logs models/checkpoints models/final data

print("✅ Directories created")
print("")
print("📁 Now upload your files using the file browser (folder icon on left):")
print("   - data_manager.py → src/")
print("   - environment.py → src/")
print("   - train_ppo.py → training/")
print("   - training.yaml → config/")
print("   - requirements_colab.txt → root")
print("   - EURUSD_M15.csv → data/")
print("   - AUDCHF_M15.csv → data/")
print("   - USDJPY_M15.csv → data/")
print("")
print("⏳ After uploading, run the next cell...")

In [ ]:
# Verify file uploads
import os

required_files = {
    'src/data_manager.py': 'Data Manager',
    'src/environment.py': 'Trading Environment',  
    'training/train_ppo.py': 'Training Script',
    'config/training.yaml': 'Training Config',
    'requirements_colab.txt': 'Requirements'
}

csv_files = {
    'data/EURUSD_M15.csv': 'EURUSD Historical Data',
    'data/AUDCHF_M15.csv': 'AUDCHF Historical Data',
    'data/USDJPY_M15.csv': 'USDJPY Historical Data'
}

print("📋 Required Files Status:")
all_uploaded = True

for file_path, description in required_files.items():
    if os.path.exists(file_path):
        size = os.path.getsize(file_path) / 1024  # KB
        print(f"✅ {description}: {file_path} ({size:.1f} KB)")
    else:
        print(f"❌ {description}: {file_path} - MISSING")
        all_uploaded = False

print("\n📊 CSV Data Files Status:")
csv_count = 0
for file_path, description in csv_files.items():
    if os.path.exists(file_path):
        size = os.path.getsize(file_path) / 1024 / 1024  # MB
        print(f"✅ {description}: {file_path} ({size:.1f} MB)")
        csv_count += 1
    else:
        print(f"⚠️ {description}: {file_path} - OPTIONAL (will use sample data)")

if all_uploaded:
    print(f"\n🎉 All required files uploaded! Found {csv_count}/3 CSV files.")
    if csv_count == 3:
        print("🎯 Ready for training with YOUR historical data!")
    else:
        print("📊 Will use sample data for missing pairs")
else:
    print("\n⚠️ Please upload missing files before continuing")

## 📦 2. Install Dependencies

**Choose ONE method below:**
- **Method 1**: Simple installation (recommended)
- **Method 2**: Advanced installation (if Method 1 fails)
- **Method 3**: Emergency fix (if both fail)

In [ ]:
# SIMPLE & RELIABLE Installation (Use this FIRST!)
print("🔧 SIMPLE INSTALLATION METHOD")
print("=" * 40)
print("This method has the highest success rate in Google Colab")

# Step 1: Clean install
print("\n🧹 Step 1: Clean previous installations...")
!pip uninstall -y stable-baselines3 gymnasium sb3-contrib -q

# Step 2: Install with specific working versions
print("\n📦 Step 2: Installing with proven working versions...")
!pip install -q gymnasium==0.29.1
!pip install -q stable-baselines3==2.0.0

# Step 3: Install additional dependencies
print("\n🔧 Step 3: Installing additional packages...")
!pip install -q torch numpy pandas pyyaml transformers

# Step 4: Test installation
print("\n🧪 Step 4: Testing installation...")
try:
    import gymnasium as gym
    import stable_baselines3 as sb3
    import torch
    
    print("✅ SUCCESS! All packages installed correctly")
    print(f"   Gymnasium: {gym.__version__}")
    print(f"   Stable-Baselines3: {sb3.__version__}")
    print(f"   PyTorch: {torch.__version__}")
    print("\n🎯 Ready to proceed with training!")
    
except ImportError as e:
    print(f"❌ Installation failed: {e}")
    print("🔄 Try the advanced installation method in the next cell")

print("\n" + "=" * 40)

In [ ]:
# Install packages with ROBUST error handling
print("📦 Installing dependencies with crash-resistant method...")
print("⚠️ This handles Colab-specific installation issues")

# Step 1: Core packages first
print("\n🔧 Step 1: Installing core ML packages...")
!pip install -q torch numpy pandas scipy matplotlib pyyaml tqdm psutil
print("✅ Core packages installed")

# Step 2: RL framework with proper error handling
print("\n🤖 Step 2: Installing RL framework (stable-baselines3)...")

# Install Gymnasium first
print("🔄 Installing Gymnasium...")
!pip install -q "gymnasium>=0.28.0,<0.30.0"
print("✅ Gymnasium installed")

# Install stable-baselines3 with multiple fallback methods
print("🔄 Installing stable-baselines3...")
try:
    # Method 1: Standard installation
    !pip install -q "stable-baselines3>=1.8.0"
    
    # Test if it actually works
    import stable_baselines3
    print("✅ stable-baselines3 installed successfully")
    
except Exception as e:
    print(f"⚠️ Standard install failed: {e}")
    print("🔄 Trying alternative installation...")
    
    # Method 2: Install specific working version
    !pip install -q stable-baselines3==2.0.0
    
    try:
        import stable_baselines3
        print("✅ stable-baselines3 v2.0.0 installed")
    except Exception as e2:
        print(f"⚠️ Version 2.0.0 failed: {e2}")
        print("🔄 Final attempt with latest...")
        
        # Method 3: Latest version with force reinstall
        !pip install --force-reinstall -q stable-baselines3
        
        try:
            import stable_baselines3
            print("✅ stable-baselines3 force installed")
        except Exception as e3:
            print(f"❌ All installation methods failed: {e3}")
            print("🔄 SOLUTION: Restart runtime (Ctrl+M .) and run this cell again")

print("\n📈 Step 3: Installing optional packages...")
optional_packages = ["transformers", "accelerate", "seaborn"]

for package in optional_packages:
    try:
        !pip install -q {package}
        print(f"✅ {package} installed")
    except:
        print(f"⚠️ {package} failed (optional)")

print("\n🎉 Installation phase complete!")
print("🔄 If stable-baselines3 failed, run the emergency fix cell below")

In [ ]:
# Install TA-Lib CAREFULLY (alternative to crashes)
print("📈 Installing TA-Lib - CRASH-RESISTANT METHOD")
print("⏳ This will take 2-3 minutes but should not crash...")

try:
    # Method 1: Try direct pip install first
    print("🔄 Trying direct pip install...")
    !pip install -q TA-Lib
    print("✅ TA-Lib installed via pip")
    
except Exception as e:
    print(f"⚠️ Direct install failed: {e}")
    
    try:
        # Method 2: Install from source if pip fails
        print("🔄 Trying source compilation...")
        !wget -q http://prdownloads.sourceforge.net/ta-lib/ta-lib-0.4.0-src.tar.gz
        !tar -xzf ta-lib-0.4.0-src.tar.gz > /dev/null 2>&1
        
        # Use background compilation to avoid hanging
        !cd ta-lib && ./configure --prefix=/usr > /dev/null 2>&1 && make > /dev/null 2>&1 && make install > /dev/null 2>&1
        !pip install -q TA-Lib
        
        print("✅ TA-Lib compiled and installed from source")
        
    except Exception as e2:
        print(f"⚠️ Source compilation failed: {e2}")
        print("📊 Will use basic indicators instead - training will still work!")

# Clean up
!rm -rf ta-lib* > /dev/null 2>&1

print("\n🎯 TA-Lib installation complete (or gracefully skipped)")

In [ ]:
# EMERGENCY FIX: Install stable-baselines3 if previous cell failed
print("🚨 EMERGENCY FIX FOR STABLE-BASELINES3")
print("=" * 50)
print("Run this cell if the previous installation failed")

try:
    # Test if stable-baselines3 is already working
    import stable_baselines3
    print("✅ stable-baselines3 already working!")
    print(f"   Version: {stable_baselines3.__version__}")
    
except ImportError:
    print("🔄 Installing stable-baselines3 with emergency method...")
    
    # Method 1: Restart and install with specific version
    print("\n🔧 Method 1: Installing specific stable version...")
    !pip uninstall -y stable-baselines3 gymnasium
    !pip install -q gymnasium==0.29.1
    !pip install -q stable-baselines3==2.0.0
    
    try:
        import stable_baselines3
        print("✅ Method 1 successful! stable-baselines3 installed")
        print(f"   Version: {stable_baselines3.__version__}")
    except ImportError:
        # Method 2: Install dependencies separately  
        print("\n🔧 Method 2: Installing dependencies separately...")
        !pip install -q torch
        !pip install -q numpy
        !pip install -q gymnasium
        !pip install -q stable-baselines3
        
        try:
            import stable_baselines3
            print("✅ Method 2 successful! stable-baselines3 installed")
        except ImportError:
            # Method 3: Manual solution
            print("\n❌ Emergency methods failed")
            print("\n🔧 MANUAL SOLUTION:")
            print("1. Go to Runtime → Restart Runtime")
            print("2. Wait for restart to complete")
            print("3. Re-run all cells from the beginning") 
            print("4. This often resolves Colab package conflicts")
            print("\n🔄 After restart, run this command first:")
            print("   !pip install gymnasium==0.29.1 stable-baselines3==2.0.0")

print("\n🎯 Emergency fix complete - try the import test cell again!")

In [ ]:
# Test imports
print("🧪 Testing imports...")

try:
    import gymnasium as gym
    import stable_baselines3
    import torch
    import transformers
    print("✅ Core RL packages imported")
    
    # Test custom modules
    import sys
    sys.path.append('src')
    
    from data_manager import DataManager
    from environment import MultiPairForexEnv
    print("✅ Custom forex modules imported")
    
    print("\n🎉 All imports successful! Ready for training.")
    
except ImportError as e:
    print(f"❌ Import error: {e}")
    print("Please check file uploads and package installation")

## 📊 3. Data Preparation & Environment Test

In [ ]:
# Test data manager and environment with CSV loading
import sys
sys.path.append('src')

from data_manager import DataManager
from environment import MultiPairForexEnv
import numpy as np
import os

print("📊 Testing data pipeline...")

# Initialize data manager
dm = DataManager()
print("✅ DataManager initialized")

# Define CSV file paths (update these to match your uploaded files)
csv_files = {
    'EURUSD': 'data/EURUSD_M15.csv',
    'AUDCHF': 'data/AUDCHF_M15.csv', 
    'USDJPY': 'data/USDJPY_M15.csv'
}

# Check which CSV files exist
print("\n📁 Checking for CSV files:")
available_files = {}
for pair, file_path in csv_files.items():
    if os.path.exists(file_path):
        size_mb = os.path.getsize(file_path) / 1e6
        print(f"✅ {pair}: {file_path} ({size_mb:.1f} MB)")
        available_files[pair] = file_path
    else:
        print(f"❌ {pair}: {file_path} - FILE NOT FOUND")

# Load data - use CSV if available, otherwise sample data
if available_files:
    print(f"\n🎯 Loading data from {len(available_files)} CSV files...")
    data = dm.get_multi_pair_data(csv_files=available_files)
    print(f"✅ Real CSV data loaded for {len(data)} pairs")
    data_source = "CSV files"
else:
    print("\n📊 No CSV files found, generating sample data...")
    data = dm.get_multi_pair_data()
    print(f"✅ Sample data generated for {len(data)} pairs")
    data_source = "Sample data"

for pair, df in data.items():
    features = len([col for col in df.columns if col != 'Symbol'])
    print(f"   {pair}: {len(df)} candles, {features} features")

# Test environment
print("\n🏗️ Testing trading environment...")
env = MultiPairForexEnv(data)
print(f"✅ Environment created using {data_source}")
print(f"   Action space: {env.action_space}")
print(f"   Observation space: {env.observation_space.shape}")

# Quick environment test
obs, info = env.reset()
action = env.action_space.sample()
obs, reward, done, truncated, info = env.step(action)
print(f"✅ Environment step test: reward={reward:.1f}, balance=${info['balance']:.2f}")

print(f"\n🎯 Ready for PPO training with {data_source}!")

## 🚀 4. PPO Training (12-24 hours)

**Target Goal**: Train AI to make **$100 profit in 4 candles**  
**Training Steps**: 500,000 timesteps  
**Expected Duration**: 12-24 hours on T4 GPU

In [ ]:
# Configure data source - YOUR CSV FILES
print("📊 DATA SOURCE CONFIGURATION")
print("=" * 40)

# IMPORTANT: Upload your CSV files to the 'data' folder first!
# Use the file browser (folder icon on left) to upload to: data/
#   - EURUSD_M15.csv
#   - AUDCHF_M15.csv  
#   - USDJPY_M15.csv

csv_files = {
    'EURUSD': 'data/EURUSD_M15.csv',
    'AUDCHF': 'data/AUDCHF_M15.csv',  
    'USDJPY': 'data/USDJPY_M15.csv'
}

# Check if files exist and show status
import os
print("📁 Checking CSV files:")
files_found = 0
total_size_mb = 0

for pair, file_path in csv_files.items():
    if os.path.exists(file_path):
        size_mb = os.path.getsize(file_path) / 1e6
        total_size_mb += size_mb
        print(f"✅ {pair}: {file_path} ({size_mb:.1f} MB)")
        files_found += 1
    else:
        print(f"❌ {pair}: {file_path} - FILE NOT FOUND")
        print(f"   📤 Please upload {pair}_M15.csv to the data/ folder")

print(f"\n📊 Status: {files_found}/3 CSV files found")
print(f"💾 Total data size: {total_size_mb:.1f} MB")

if files_found == 3:
    print("🎉 All CSV files ready! Training will use YOUR historical data")
    print("🎯 AI will learn from REAL 10-year market patterns")
    print("📈 100,000+ candles per pair = Excellent training dataset")
elif files_found > 0:
    print(f"⚠️ Partial data available - will train on {files_found} pairs")
else:
    print("⚠️ No CSV files found - will use sample data for testing")
    print("📤 To use your data:")
    print("   1. Click folder icon on left sidebar")  
    print("   2. Create 'data' folder if needed")
    print("   3. Upload your CSV files to data/ folder")
    print("   4. Refresh and re-run this cell")

print(f"\n🔧 Ready to proceed with {'CSV data' if files_found > 0 else 'sample data'}")

In [ ]:
# Start PPO training with CSV data
print("🚀 STARTING PPO TRAINING FOR FOREX SYSTEM")
print("=" * 50)
print("🎯 Goal: $100 profit in 4 candles (15-min timeframe)")
print("⏰ Expected time: 12-24 hours")
print("📊 Total timesteps: 500,000")

# Check for CSV files
import os
csv_files = {
    'EURUSD': 'data/EURUSD_M15.csv',
    'AUDCHF': 'data/AUDCHF_M15.csv', 
    'USDJPY': 'data/USDJPY_M15.csv'
}

available_csv_files = {}
for pair, file_path in csv_files.items():
    if os.path.exists(file_path):
        size_mb = os.path.getsize(file_path) / 1e6
        print(f"✅ Found {pair}: {file_path} ({size_mb:.1f} MB)")
        available_csv_files[pair] = file_path

if available_csv_files:
    print(f"\n🎉 Training on YOUR {len(available_csv_files)} CSV files!")
    for pair, file_path in available_csv_files.items():
        print(f"   {pair}: {file_path}")
    data_source = available_csv_files
else:
    print("\n📊 No CSV files found - training on sample data")
    data_source = None

print("\n⚠️ Do NOT close this tab during training!")
print("\n" + "=" * 50)

# Import and run training
import sys
sys.path.append('training')
from train_ppo import main

# Start training with CSV files (if available)
main(data_source)

In [ ]:
# Alternative: Start PPO training with %run command
print("🚀 STARTING PPO TRAINING FOR FOREX SYSTEM")
print("=" * 50)
print("🎯 Goal: $100 profit in 4 candles (15-min timeframe)")
print("⏰ Expected time: 12-24 hours")
print("📊 Total timesteps: 500,000")
print("\n⚠️ Do NOT close this tab during training!")
print("\n" + "=" * 50)

# Run the training script
%run training/train_ppo.py

## 📈 5. Training Monitoring (Run during training)

In [ ]:
# Launch TensorBoard (run this in a separate cell while training)
%load_ext tensorboard
%tensorboard --logdir logs/tensorboard

print("📊 TensorBoard launched above")
print("📈 Monitor training progress, rewards, and loss curves")
print("🎯 Look for 'forex/goal_trades_per_rollout' to track success!")

In [ ]:
# Check training progress (run periodically)
import os
import glob
from datetime import datetime

print(f"🕐 Current time: {datetime.now().strftime('%H:%M:%S')}")

# Check for model checkpoints
checkpoints = glob.glob('models/checkpoints/*.zip')
if checkpoints:
    latest = max(checkpoints, key=os.path.getctime)
    mod_time = datetime.fromtimestamp(os.path.getmtime(latest))
    print(f"💾 Latest checkpoint: {latest}")
    print(f"   Created: {mod_time.strftime('%H:%M:%S')}")
else:
    print("💾 No checkpoints yet...")

# Check log files
if os.path.exists('logs/evaluations'):
    eval_files = os.listdir('logs/evaluations')
    print(f"📊 Evaluation files: {len(eval_files)}")

print("\n🔄 Training is running... Check TensorBoard above for real-time metrics!")

## 💾 6. Download Trained Model (After training completes)

In [ ]:
# Download final trained model
import os
from google.colab import files
import zipfile

print("📦 Preparing model for download...")

# Create download package
download_files = [
    'models/final/forex_ppo_model.zip',
    'config/training.yaml'
]

# Check if training completed
model_path = 'models/final/forex_ppo_model.zip'
if os.path.exists(model_path):
    size_mb = os.path.getsize(model_path) / 1e6
    print(f"✅ Final model ready: {model_path} ({size_mb:.1f} MB)")
    
    # Download the model
    files.download(model_path)
    
    # Download config
    if os.path.exists('config/training.yaml'):
        files.download('config/training.yaml')
    
    print("\n🎉 Model downloaded successfully!")
    print("📁 Files downloaded:")
    print("   - forex_ppo_model.zip (trained model)")
    print("   - training.yaml (configuration)")
    
else:
    print("❌ Model not found. Training may not be complete yet.")
    
    # Check for latest checkpoint
    import glob
    checkpoints = glob.glob('models/checkpoints/*.zip')
    if checkpoints:
        latest = max(checkpoints, key=os.path.getctime)
        print(f"💾 Latest checkpoint available: {latest}")
        download_latest = input("Download latest checkpoint? (y/n): ")
        if download_latest.lower() == 'y':
            files.download(latest)

## 📊 7. Training Results Summary

In [ ]:
# Training results summary
print("🏆 FOREX TRADING SYSTEM TRAINING COMPLETE!")
print("=" * 50)
print("🎯 Goal: AI that makes $100 profit in 4 candles")
print("🤖 Algorithm: PPO with custom reward function")
print("📊 Training: 500k timesteps on GPU")
print("\n📈 Next Steps:")
print("1. Download the trained model (forex_ppo_model.zip)")
print("2. Use it locally for Phase 4: Inference Engine")
print("3. Build the desktop UI for live trading")
print("\n🎉 Your AI forex trader is ready!")